In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [3]:
f = 'variants_withinexonsOrSS_sampleinfo_included.tsv'
df = pd.read_csv(f, sep='\t')

In [4]:
# only novel exons
df = df.loc[df.novelty.isin(['Novel', "Novel 5'/3'"])]

In [9]:
# only yri exons
# df = df.loc[df.YRI>=1]
meta = load_meta()
meta = meta.loc[meta.merged_run_mode==True]
meta = meta.loc[meta.mixed_samples == False]
yri_cols = meta.loc[meta.population=='YRI', 'sample'].unique().tolist()
df = df.loc[df[yri_cols].sum(axis=1)>=1]

In [11]:
# sort by samples reproducible in 
# df = df.sort_values(by='YRI', ascending=False)

In [12]:
# get only high fst things
df = df.loc[df.highfst=='fst>=0.25']
df.head()

,contig,position,fst,novelty,ITU5,ITU4,ITU3,ITU2,ITU1,PEL6,...,CEU4,CEU3,CEU2,CEU1,MPC4,MPC3,MPC2,MPC1,splice_site_variant,highfst
486,chr10,7466155,0.319106,Novel,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,False,fst>=0.25
1734,chr10,45994680,0.297797,Novel,1,0,0,1,1,1,...,0,1,0,0,0,0,0,1,False,fst>=0.25
2207,chr10,65837949,0.413240,Novel,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,False,fst>=0.25
2386,chr10,69097096,0.502111,Novel 5'/3',0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,False,fst>=0.25
3009,chr10,78034315,0.325580,Novel,0,1,0,0,1,0,...,1,1,0,0,1,1,1,0,False,fst>=0.25


In [13]:
# in splice site things only
df = df.loc[df.splice_site_variant==True]

In [14]:
df.head()

,contig,position,fst,novelty,ITU5,ITU4,ITU3,ITU2,ITU1,PEL6,...,CEU4,CEU3,CEU2,CEU1,MPC4,MPC3,MPC2,MPC1,splice_site_variant,highfst
26428,chr13,48080390,0.559142,Novel,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,True,fst>=0.25
45512,chr16,67935628,0.358167,Novel 5'/3',0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,True,fst>=0.25


In [15]:
df[yri_cols]

,YRI7,YRI6,YRI5,YRI3,YRI2,YRI1
26428,0,0,0,1,0,0
45512,0,1,0,0,0,0


In [17]:
exon_df = pd.read_csv('241126_exons_novelty_sample_det.tsv', sep='\t')
exon_df['contig'] = exon_df.eid.str.split('_', expand=True)[0]
exon_df['contig'] = exon_df.eid.str.split('_', expand=True)[0]

In [18]:
exon

,Unnamed: 0,eid,novelty,ITU5,ITU4,ITU3,ITU2,ITU1,PEL6,PEL5,...,YRI1,CEU5,CEU4,CEU3,CEU2,CEU1,MPC4,MPC3,MPC2,MPC1
0,0,GL000008.2_+_135133_135173_,Known,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,GL000008.2_+_155429_155531_,Known,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,GL000008.2_+_173515_173643_,Known,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,GL000008.2_+_83859_84014_,Known,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,GL000008.2_+_83926_84014_,Known,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
